# Data preprocessing for CNN

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

Extract "# of weeks" for each customer

In [ ]:
# reader = pd.read_csv('../../data/pnc/inbound/sample_1_not_offered_data.csv', chunksize=100000)
# data = None
# keys_all = set()
# for chunk in reader:
#     items = pd.groupby(chunk, by='cif_permanent_key').count()['week'].reset_index()
#     if items.loc[0,'cif_permanent_key'] in keys_all:
#         items.loc[0,'week'] += data.week.iloc[-1]
#         data = data[:-1]
#     keys_all |= set(items['cif_permanent_key'])
#     data = pd.concat([data, items])
    
# data_cross = pd.read_csv('cross_data.csv')
# data_comb = pd.merge(data, data_cross.loc[:,['cif_permanent_key', 'apply']], on='cif_permanent_key')
# data_comb.to_csv('data_count.csv', index=False)

In [3]:
data_cross = pd.read_csv('cross_data.csv')
data_count = pd.read_csv('data_count.csv')
data_sample = pd.read_csv('../../../data/pnc/inbound/sample_1_not_offered_data.csv', nrows=10000)

In [4]:
pos_key = list(data_count.cif_permanent_key[data_count.eval('week >= 52 and apply == 1')])
neg_key = list(data_count.cif_permanent_key[data_count.eval('week >= 52 and apply == 0')])

Use all eligible positive customers and randomly select negative customers.

In [5]:
total = 10000
np.random.seed(1234)
sel_neg_key = list(np.random.choice(neg_key, total-len(pos_key), replace=False))
sel_key = pos_key + sel_neg_key
len(pos_key), len(sel_neg_key), len(pos_key)+len(sel_neg_key)

(2739, 7261, 10000)

Extract the first 52 weeks' data for the chosen customers

In [ ]:
# reader = pd.read_csv('../../data/pnc/inbound/sample_1_not_offered_data.csv', chunksize=100000)
# data = None
# keys_all = set()
# for chuck in reader:
#     items = chuck[[key in sel_key and week <=52 for key, week in zip(chuck.cif_permanent_key, chuck.week)]]
#     try:
#         data = pd.concat([data, items])
#     except:
#         data = items
#     del items
# data.to_csv('data_52w.csv', index=False)

In [6]:
data = pd.read_csv('data_52w.csv')

In [7]:
data_count = pd.read_csv('data_count.csv')
key_ind = [k in sel_key for k in data_count.cif_permanent_key]
y = data_count.loc[key_ind,'apply'].values

In [8]:
categorical_var = ['age_grp_4L', 'hh_agr_type', 'inc_grp_3L', 'lifestage', 'segment']
date_var = ['date', 'time_period', 'date_applied', 'week', 'week_applied', 'MLK', 'Presidents', 'Memorial', 
            'Independence', 'Labor', 'Colombus', 'Veterans', 'Thanksgiving', 'Christmas', 'NewYears']
key_var = ['rlb_location_key']
others = ['offered']
groupby = ['cif_permanent_key', 'month', 'year', 'apply']

data = pd.get_dummies(data, columns=categorical_var, prefix_sep=['_' for col in categorical_var])
features_std = list(data.columns)
for f in date_var + key_var + others + groupby:
    features_std.remove(f)
    
X = data[features_std] 
X.rename(columns={'segment_Lower Mas':'segment_Lower', 'segment_Mass Affl':'segment_Mass', 'segment_Upper Mas':'segment_Upper',
                  'age_grp_4L_a [18 - 25]':'age_grp_4L_a', 'age_grp_4L_b [26 - 45]':'age_grp_4L_b', 'age_grp_4L_c [46 - 65]':'age_grp_4L_c',
                  'age_grp_4L_d [66 - 100]':'age_grp_4L_d', 'inc_grp_3L_a [0 - 50k)':'inc_grp_3L_a', 'inc_grp_3L_b [50k - 125k)':'inc_grp_3L_b',
                  'inc_grp_3L_c [125k - inf)':'inc_grp_3L_c'}, inplace=True)
    
features = X.columns
std_scale = StandardScaler().fit(X)
X_std = std_scale.transform(X)
X_std_all = pd.concat([pd.DataFrame(X_std, columns=features), data[groupby]], axis=1)

/opt/anaconda3/lib/python3.5/site-packages/pandas/core/frame.py:2834: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  **kwargs)


Reshape to 3D data (customers x months x features = 10000 x 13 x 193)

In [16]:
features = list(X_std_all.columns)
month_str = pd.Series([i.zfill(2) for i in list(X_std_all.month.apply(str))])
X_std_all['ym'] = X_std_all.year.apply(str) + '+' + month_str

features.remove('month')
features.remove('year')
features.remove('apply')
try: 
    features.remove('ym')
except: pass

In [19]:
group = pd.groupby(X_std_all, by=['cif_permanent_key','ym']).apply(np.max)
group_mouth = pd.DataFrame(group[features].as_matrix(), columns=features)

In [ ]:
group

In [50]:
a3d = group_month.groupby('cif_permanent_key').apply(pd.DataFrame.as_matrix).apply(lambda x: x[:,:-1])

In [54]:
X = np.stack(a3d)
cust_no = X.shape[0]

X_ind = np.array(range(cust_no*13)).reshape(cust_no,13)

In [91]:
np.save('X.npy', X) 
np.save('y.npy', y)

In [85]:
trn_X, tst_X, trn_y, tst_y = train_test_split(X, y, test_size=0.3, random_state=12)
print(sum(trn_y), sum(tst_y))

1918 821
